# DIY - Exercises


* Do it Yourself Exercises through the course

## Section 3 - DataFrames and RDDs

1. Query name, state, status

2. Query clients with status "platinum" and "gold"

3. Query sales amount per client status

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("diy_section_3").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/15 08:35:43 WARN Utils: Your hostname, WIN-NJTBBD8GS0T, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/09/15 08:35:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/15 08:35:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
clients = spark.read.parquet("/home/ubuntu/obenkyo/raw_data/spark_course_udemy/Atividades/Clientes.parquet")

In [ ]:
clients.select("Cliente", "Estado", "Status").show(10)

+--------------------+------+--------+
|             Cliente|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
+--------------------+------+--------+
only showing top 10 rows


In [19]:
from pyspark.sql.functions import col

(
    clients
    .filter(col("Status").isin("Platinum", "Gold"))
    .orderBy(col("Status"), col("ClienteID").desc())
    .show()
)

+---------+-------------------+------+------+--------+
|ClienteID|            Cliente|Estado|Genero|  Status|
+---------+-------------------+------+------+--------+
|      237|  Iracema Rodríguez|    BA|     F|    Gold|
|      220|Honorina Villaverde|    PE|     F|    Gold|
|      166|   Firmino Meireles|    AM|     M|    Gold|
|      121|    Dionísio Saltão|    PR|     M|    Gold|
|       83|      Cláudio Jorge|    TO|     M|    Gold|
|       68|      Carminda Dias|    AM|     F|    Gold|
|       49|      Bento Quintão|    SP|     M|    Gold|
|       28|      Anna Carvajal|    RS|     F|    Gold|
|       10|   Alberto Monsanto|    RN|     M|    Gold|
|      247|         Joana Ataí|    GO|     F|Platinum|
|      230|    Ibijara Botelho|    RR|     F|Platinum|
|      170|      Flor Vilanova|    CE|     M|Platinum|
|        4|   Adriana Guedelha|    RO|     F|Platinum|
+---------+-------------------+------+------+--------+



In [37]:
from pyspark.sql import functions as Func

(
    clients
    .select("*")
    .where((Func.col("Status") == "Gold") | (Func.col("Status") == "Platinum"))
    .show()
)

+---------+-------------------+------+------+--------+
|ClienteID|            Cliente|Estado|Genero|  Status|
+---------+-------------------+------+------+--------+
|        4|   Adriana Guedelha|    RO|     F|Platinum|
|       10|   Alberto Monsanto|    RN|     M|    Gold|
|       28|      Anna Carvajal|    RS|     F|    Gold|
|       49|      Bento Quintão|    SP|     M|    Gold|
|       68|      Carminda Dias|    AM|     F|    Gold|
|       83|      Cláudio Jorge|    TO|     M|    Gold|
|      121|    Dionísio Saltão|    PR|     M|    Gold|
|      166|   Firmino Meireles|    AM|     M|    Gold|
|      170|      Flor Vilanova|    CE|     M|Platinum|
|      220|Honorina Villaverde|    PE|     F|    Gold|
|      230|    Ibijara Botelho|    RR|     F|Platinum|
|      237|  Iracema Rodríguez|    BA|     F|    Gold|
|      247|         Joana Ataí|    GO|     F|Platinum|
+---------+-------------------+------+------+--------+



In [20]:
sales = spark.read.parquet("/home/ubuntu/obenkyo/raw_data/spark_course_udemy/Atividades/Vendas.parquet")

In [ ]:
client

In [38]:
from pyspark.sql.functions import sum as _sum

sales.join(clients, on=["ClienteID"], how="inner").groupBy("Status").agg(_sum("Total")).show()

+--------+------------------+
|  Status|        sum(Total)|
+--------+------------------+
|Platinum|          12584.68|
|  Silver|        3014291.36|
|    Gold|27286.690000000002|
+--------+------------------+



## Section 4 - Spark SQL

1. Create a DW called Vendas Varejo

2. Create a query showing: client name, sale date, sku, seller, total value

In [7]:
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder.appName("section_4_exs").getOrCreate()

### Exercise 1

In [8]:
base_path = "/home/ubuntu/obenkyo/raw_data/spark_course_udemy/Atividades/"
clientes = spark.read.parquet(os.path.join(base_path, "Clientes.parquet"))
vendas = spark.read.parquet(os.path.join(base_path, "Vendas.parquet"))
item_vendas = spark.read.parquet(os.path.join(base_path, "ItensVendas.parquet"))
produtos = spark.read.parquet(os.path.join(base_path, "Produtos.parquet"))
vendedores = spark.read.parquet(os.path.join(base_path, "Vendedores.parquet"))

In [11]:
clientes.write.mode("overwrite").saveAsTable("diy_4_clientes")
vendas.write.mode("overwrite").saveAsTable("diy_4_vendas")
item_vendas.write.mode("overwrite").saveAsTable("diy_4_item_vendas")
produtos.write.mode("overwrite").saveAsTable("diy_4_produtos")
vendedores.write.mode("overwrite").saveAsTable("diy_4_vendedores")

### Exercise 2

In [15]:
spark.sql(
    """
    SELECT
        ClienteID AS client_id,
        Data AS sales_date,
        Produto AS product_name,
        Vendedor AS salesman_name,
        ValorTotal AS total_value
    FROM diy_4_vendas
    LEFT JOIN diy_4_clientes
    USING(ClienteID)
    LEFT JOIN diy_4_item_vendas
    USING(VendasID)
    LEFT JOIN diy_4_produtos
    USING(ProdutoID)
    LEFT JOIN diy_4_vendedores
    USING(VendedorID)
    """
).show(10)

+---------+----------+--------------------+----------------+-----------+
|client_id|sales_date|        product_name|   salesman_name|total_value|
+---------+----------+--------------------+----------------+-----------+
|       91|  1/1/2019|Bicicleta Altools...|    Armando Lago|    7820.85|
|       91|  1/1/2019|Bermuda Predactor...|    Armando Lago|      97.75|
|       91|  1/1/2019|Camiseta Predacto...|    Armando Lago|      135.0|
|      185|  1/1/2020|Luva De Ciclismo ...|   Iberê Lacerda|      150.4|
|       31|  2/1/2020|Capacete Gometws ...|Jéssica Castelão|      155.0|
|       31|  2/1/2020|Bicicleta Gometws...|Jéssica Castelão|     5932.0|
|       31|  2/1/2019|Bicicleta Altools...|  Hélio Liberato|    7820.85|
|       31|  2/1/2019|Bermuda Predactor...|  Hélio Liberato|      97.75|
|       31|  2/1/2019|Bicicleta Gometws...|  Hélio Liberato|     5910.0|
|       31|  3/1/2018|Bicicleta Gometws...|  Hélio Liberato|     2955.0|
+---------+----------+--------------------+--------

# Section 7 - Machine Learning

In [21]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder.appName("section_7_exs").getOrCreate()

In [4]:
dataset = spark.read.csv(
    "/home/ubuntu/obenkyo/raw_data/spark_course_udemy/iris.csv",
    schema="sepallength FLOAT, sepalwidth FLOAT, petallength FLOAT, petalwidth FLOAT, class STRING",
    sep=",",
    header=True
)

In [28]:
# Preprocess features
assembler = VectorAssembler(
    inputCols=[("sepallength"), ("sepalwidth"), ("petallength"), ("petalwidth")], outputCol="features"
)

# Preprocess label
label_indexer = StringIndexer(inputCol="class", outputCol="label", handleInvalid="keep")

# Model
logReg = LogisticRegression(featuresCol="features", labelCol="label")


In [29]:
pipeline = Pipeline(
    stages=[
        assembler,
        label_indexer,
        logReg
    ]
)

In [36]:
train, test = dataset.randomSplit([0.7, 0.3], seed=42)

In [37]:
model = pipeline.fit(train)
pred = model.transform(test)

In [38]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

In [39]:
evaluator.evaluate(pred)

0.9782608695652174